#Unet

The following cells create a class called Unet, responsible for holding the CNN model, which will be used to classify the images.

In [ ]:
# Import necessary libraries
from keras.models import Model
from keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, 
    Conv2DTranspose, Dropout, BatchNormalization, UpSampling2D, Lambda)
import tensorflow as tf

In [ ]:
# Creates class
class Unet:
    def __init__(self, outchns=1):
        self.outchns = outchns
        self.model = self.build()
        
    def loadWeights(self, pathToWeights):
        try:
            self.model.load_weights(pathToWeights)
        except:
            print(pathToWeights, 'cannot be loaded')

    def convolutionLayer(self, num_of_filters, image):
        conv = Conv2D(num_of_filters, (3, 3), activation='relu', padding='same')(image)
        conv = BatchNormalization()(conv)
        conv = Conv2D(num_of_filters, (3, 3), activation='relu', padding='same')(conv)
        conv = BatchNormalization()(conv)
        return conv
        
    def build(self):
        # Defines the input that the model will take
        # This is ususally the dimensions of the image
        inputs = Input((160, 160, 3))

        # Sets a list of convolutions and pooling.
        # Check the following article for a better understanding
        # https://towardsdatascience.com/understanding-convolutions-and-pooling-in-neural-networks-a-simple-explanation-885a2d78f211
        # DRY codes
        layers = {}
        pool = inputs

        for i in range(5, 9):
          conv = Unet.convolutionLayer(self, 2**i, pool)
          pool = MaxPooling2D(pool_size=(2, 2))(conv)
          layers[f'conv{i}'] = conv

        conv = Unet.convolutionLayer(self, 2**9, pool)
        conv = Dropout(0.2)(conv)
        layers['conv9'] = conv

        for i in range(8, 4, -1):
          up = concatenate([Conv2DTranspose(2**i, (3, 3), strides=(2, 2), padding='same')(layers[f'conv{i+1}']), layers[f'conv{i}']], axis=3)
          conv = Unet.convolutionLayer(self, 2**i, up)
          layers[f'conv{i+1}'] = conv
        
        conv = Conv2D(self.outchns, (1, 1), activation='sigmoid')(layers['conv6'])
        model = Model(inputs=[inputs], outputs=[conv])

        return model

#Config file

This part of the code set the parameters to be used in other pieces of the code. The parameters are: Batch size, image resolution, number of epochs, data augmentation parameters for training and validation set.

In [ ]:
import albumentations as A

class config:
  BATCH_SIZE = 30
  IMG_SIZE = 160

  CLASSES = 1  # number of output channels

  EPOCHS = 50

  # The following is used to augment the images of the dataset
  AUGMENTER_TRAIN = A.Compose(
      transforms=[
          A.Resize(height=IMG_SIZE, width=IMG_SIZE, always_apply=True, p=1.0),
          # A.RandomCrop(height=IMG_SIZE, width=IMG_SIZE, always_apply=True, p=1.0),
          A.Transpose(p=0.5),
          A.VerticalFlip(p=0.5),
          A.HorizontalFlip(p=0.5),
      ],
  )

  AUGMENTER_VAL = A.Compose(
      transforms=[
          A.Resize(height=IMG_SIZE, width=IMG_SIZE, always_apply=True, p=1.0),
          # A.CenterCrop(height=IMG_SIZE, width=IMG_SIZE, always_apply=True, p=1.0),
      ],
  )

#Dataset

Here, we are creating function that will split the data between training and testing sets, as well as implementing the data augmentation to our dataset.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import Iterator


def random_image_and_mask(config):
    S = config.IMG_SIZE
    img = np.zeros(shape=(S, S, 3), dtype=np.uint8)
    mask = np.zeros(shape=(S, S, config.CLASSES), dtype=np.uint8)

    # Random color for img
    img += np.random.randint(0, 255, size=(3,), dtype=np.uint8)

    # Add a random box
    w, h = np.random.randint(int(S * 0.1), int(S * 0.3), size=(2,))
    box_color = np.random.randint(0, 255, size=(3,), dtype=np.uint8)
    x, y = np.random.randint(0, int(S * 0.9), size=(2,))
    img[y:y+h, x:x+w, :] = box_color
    mask[y:y+h, x:x+w, :] = 255

    return img, mask


class IteratorWithAug(Iterator):
    """ Iterator that generate data from directory and a list of images and a
        corresponding list of class labels
    """

    def __init__(self,
                 image_paths,
                 mask_paths,
                 config,
                 augmenter=None,
                 mode=None,
                 shuffle=True,
                 batch_size=16,
                 seed=None,
                 ):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.n = len(image_paths)
        #assert self.n == len(mask_paths)
        self.batch_size = batch_size
        self.mode = mode
        self.config = config
        self.augmenter = augmenter

        super().__init__(self.n,
                         batch_size,
                         shuffle,
                         seed=seed)

    def _get_batches_of_transformed_samples(self, index_array):
        # build batch of image & gt
        batch_x = []
        batch_y = []
        for i, j in enumerate(index_array):
            # TODO: read rgb from: self.image_paths[j]
            # TODO: read bw mask from: self.mask_paths[j]
            rgb, bw = random_image_and_mask(self.config)

            # Run Augmentation
            if self.augmenter is not None:
                transformed = self.augmenter(image=rgb, mask=bw, keypoints=[])
                transformed_image = transformed['image']
                transformed_mask = transformed['mask']
            else:
                transformed_image = rgb
                transformed_mask = bw

            if len(transformed_mask.shape) == 2:
                transformed_mask = transformed_mask[..., np.newaxis]

            batch_x.append(transformed_image)
            batch_y.append(transformed_mask)

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)

        # standardize
        batch_x = (batch_x.astype(np.float32) / 255) - 0.5
        batch_y = (batch_y > 127).astype(np.float32)

        return batch_x, batch_y

# Train

With all the functions declared and parameters set, we can train the CNN model on the dataset.

In [ ]:
# COLAB ONLY: Check for GPU environment
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import os
from datetime import datetime
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras import backend as K

In [ ]:
# Function to calculate the jaccard distance:
# https://opensourcebiology.eu/2021/09/22/how-to-understand-this-jaccard-distance/
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
# Set up datasets (currently dummy data)
x_train = [''] * 10000  # TODO: not used for this demo of random images... Later, set to file paths
y_train = [''] * 10000  # TODO: not used for this demo of random images... Later, set to file paths
x_val = [''] * 100  # TODO: not used for this demo of random images... Later, set to file paths
y_val = [''] * 100  # TODO: not used for this demo of random images... Later, set to file path

This dataset is made of emptty strings and it is only used to test the code's execution. When using real data, the varibles need to be set to a directory path, like:

In [ ]:
# Set up datasets (sample)
x_train = 'data/Training_Input'
y_train = 'data/Training_GroundTruth/pigment_network'
x_val = 'data/Validation_Input' 
y_val = 'data/Validation_GroundTruth/pigment_network' 

In [ ]:
# Used to apply data augmentation to the data
train_generator = IteratorWithAug(
    image_paths=x_train,
    mask_paths=y_train,
    config=config,
    augmenter= config.AUGMENTER_TRAIN,
    mode='train',
    shuffle=True,
    batch_size= config.BATCH_SIZE
)

val_generator = IteratorWithAug(
    image_paths=x_val,
    mask_paths=y_val,
    config=config,
    augmenter= config.AUGMENTER_VAL,
    mode='val',
    shuffle=False,
    batch_size= config.BATCH_SIZE
)

In [ ]:
# Set up log dir
datetime_str = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
LOG_DIR = os.path.join(r'.\logs', datetime_str)

In [ ]:
# Build Model
net = Unet(outchns= config.CLASSES)
model = net.model

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss='mean_squared_error',
    metrics=[
        jaccard_distance
    ]
)

In [ ]:
# Train
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs= 25,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[
        ModelCheckpoint(os.path.join(LOG_DIR, 'weights_{epoch:02d}.h5'), save_weights_only=False),
        TensorBoard(LOG_DIR, write_graph=False)
    ]
)

Epoch 1/25
1/1 [==============================] - 8s 8s/step - loss: 0.3128 - jaccard_distance: 0.4950 - val_loss: 0.6983 - val_jaccard_distance: 0.6919
Epoch 2/25


KeyboardInterrupt: ignored

# Setup for using the actual data

We use the following 2 cells to mount the colab with our drive.
Once mounted, set the path to be equal to where you have the data stored in your drive.

Once the data is loaded, you can edit the x and y values above to be equal to the path where you have the training and testing datasets.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.getcwd()
path = '/content/drive/My Drive/CanField'
os.chdir(path)

## Classify single image

Once the model is trained on real data, we can use it to classify different images.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [ ]:
img = mpimg.imread('data/Validation_Input/ISIC_0012585.jpg')
plt.imshow(img)
plt.show()

In [ ]:
import cv2
img_array = cv2.imread('data/Validation_Input/ISIC_0012585.jpg')
new_array = cv2.resize(img_array, (160, 160))
final_img = new_array.reshape(-1, 160, 160, 3)
prediction = model.predict([final_img])
print(prediction)

In [ ]:
count_zeros = 0
for i in prediction:
  for j in i:
    if j[0] == 0:
      count_zeros += 1
  
print(count_zeros)

In [ ]:
print(prediction.shape)

In [ ]:
from PIL import Image

data = np.reshape(prediction, (160, 160))
img = Image.fromarray(data, '1')
img.save('test2.png')
img.show()